In [ ]:
"""
Code Description:
This script performs the following tasks to consolidate and manage IM (Intensity Measure) data 
from existing and new realizations:

1. **Data Consolidation**:
   - Combines all IM values from existing and new realizations into a newly generated combined directory.
   - Identifies common stations and IM types for each source (fault) across old and new realizations.
   - Filters out uncommon stations and IM types to ensure consistency.
   ---Note: Due to Nan and integer filteration lines added, different realizations may have different number of stations.
      This happened in for subduction faults realizations in v24.9
      

2. **Median Calculation**:
   - Computes the median IM values for each station and IM type from all realizations.
   - Saves the computed median values as a CSV file.

3. **Source File Management**:
   - Copies all "Source" files from old and new realizations into the "Source" directory in the combined folder.
   - Copies `nzvm.cfg` and `vm_params.yaml` files from the old realizations as-is.
   - Copies these files from the new realizations with a "_new" suffix added to the filenames for differentiation.

4. **Summary Reports**:
   - Generates summary reports, including an Excel file, that details the number of stations and IMs processed.
   - Adds a note file providing context and metadata about the combined data.

Author: Morteza
Version History:
- Version 1.0: August 18, 2024
- Version 1.1: December 18, 2024

To Do:
 - in the "Merge IMs" section, first load each realization's IM values in a dataarray, then concatenate them "inner". This will 
   ensure that only common stations and IM types are retained across all realizations. Then save csv files.

"""

# Import the dependencies

In [34]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm

import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr

# Path and directory works

In [35]:
file_path = Path.cwd()
root_path = str(file_path.parent.parent)

versions = [
    # "v21p1",
    "v21p6",
    "v21p6p2",
    "v22p2",
    "v22p4",
    "v23p5",
    "v23p7",
    "v23p10",
]
Comp = "rotd50" # "000", "090", "geom", "rotd100_50", "rotd50", "ver"

vers_path = {
    v: os.path.join(root_path, "Cybershake_Data", v) for v in versions
}

combined_cs_files_path = os.path.join(
    root_path, "Cybershake_Data", "combined_cs100"
)

log_file_path = combined_cs_files_path
log_file = os.path.join(combined_cs_files_path, 'combination_logs.txt')

# Check if the directory exists
if Path(combined_cs_files_path).exists():
    prompt = input(
        f"The following path already exists! Do you want to delete and renew (1) or terminate (2)? \n{combined_cs_files_path}\n"
    )
    if prompt == "1":
        # Remove the folder
        shutil.rmtree(combined_cs_files_path)
        print(f"Deleted and renewed the path: \n{combined_cs_files_path}")
        os.makedirs(combined_cs_files_path, exist_ok=False)
    elif prompt == "2":
        print("Terminating the process.")
        exit()
    else:
        print("Invalid input. Terminating the process.")
        exit()
else:
    os.makedirs(combined_cs_files_path, exist_ok=False)
    print(f"Created the path: \n{combined_cs_files_path}")



Created the path: 
/mnt/hypo_data/mab419/cs100/Cybershake_Data/combined_cs100


# Merge Files

In [36]:
def strip_and_convert(value):
    """
    Strips leading/trailing whitespace or single quotes from a value
    and converts it to a float if possible.
    """
    try:
        return float(str(value).strip("'"))
    except ValueError:
        return value  # Return as-is if it can't be converted
    
def log_and_print(log_lines: list, msg: str, print = False):
    if print:
        print(msg)
    log_lines.append(msg)
    
log_lines = []
    
im_true_order = [
    "PGA",
    "PGV",
    "CAV",
    "AI",
    "Ds575",
    "Ds595",
    "MMI",
    "pSA_0.01",
    "pSA_0.02",
    "pSA_0.03",
    "pSA_0.04",
    "pSA_0.05",
    "pSA_0.075",
    "pSA_0.1",
    "pSA_0.12",
    "pSA_0.15",
    "pSA_0.17",
    "pSA_0.2",
    "pSA_0.25",
    "pSA_0.3",
    "pSA_0.4",
    "pSA_0.5",
    "pSA_0.6",
    "pSA_0.7",
    "pSA_0.75",
    "pSA_0.8",
    "pSA_0.9",
    "pSA_1.0",
    "pSA_1.25",
    "pSA_1.5",
    "pSA_2.0",
    "pSA_2.5",
    "pSA_3.0",
    "pSA_4.0",
    "pSA_5.0",
    "pSA_6.0",
    "pSA_7.5",
    "pSA_10.0",
]    

vers_faults = {
    v: [cur_dir.stem for cur_dir in Path(vers_path[v]).iterdir() if cur_dir.is_dir()]
    for v in versions
}

all_faults = list(set().union(*vers_faults.values()))

summary_dat_columns = [col for v in versions for col in (f"n_rels_{v}", f"n_stations_{v}", f"n_IMs_{v}")] + ["n_rel_combined", "n_stations_combined", "n_IMs_combined"]
summary_data = pd.DataFrame(index=all_faults, columns=summary_dat_columns)

##########################################################################
########################### working on IM data ###########################
##########################################################################

for cur_fault in tqdm(all_faults):
    cur_fault_versions = [
        v for v in versions if cur_fault in vers_faults[v]
    ]
    cur_fault_versions.sort()
    IM_DATA = {v: None for v in cur_fault_versions}
    for v in cur_fault_versions:
        # identify existing relaizations
        fault_version_realizations = [rel for rel in Path(Path(vers_path[v]) / cur_fault / "IM").rglob("*REL*.csv")]
        fault_version_realizations.sort()
        
        # write number of realizations to summary data
        summary_data.loc[cur_fault, f"n_rels_{v}"] = len(fault_version_realizations)
        
        REL_DATA = {cur_im_file.stem: None for cur_im_file in fault_version_realizations}
        file_counter = 0
        for cur_im_file in fault_version_realizations:
            # Read the current IM file
            with open(cur_im_file, 'r') as f:
                header = f.readline().strip().split(',')

            converters = {
                col: strip_and_convert for col in range(2, len(header))
            }

            cur_im_df = pd.read_csv(
                cur_im_file,
                converters=converters
            )
            
                     
            cur_im_df.columns = [
                (
                    col[:5] + col[5:].replace("p", ".", 1)
                    if col.startswith("pSA_") and "p" in col[5:]
                    else col
                )
                for col in cur_im_df.columns
            ]
            
            # writing summary of data assuming that all realizations in a version has the same stations and IMs
            if file_counter == 0:
                summary_data.loc[cur_fault, f"n_stations_{v}"] = len(set(cur_im_df['station']))
                summary_data.loc[cur_fault, f"n_IMs_{v}"] = len(list(set(cur_im_df.columns) - {'station', 'component'}))
                file_counter += 1
            
            # Filter non-valid IMs
            temp_df1 = cur_im_df.copy()
            temp_df1.iloc[:, 0] = temp_df1.iloc[:, 0].astype(str)
            temp_df1.iloc[:, 1] = temp_df1.iloc[:, 1].astype(str)

            float_cols = temp_df1.columns[2:]
            temp_df1.loc[:, float_cols] = temp_df1.loc[:, float_cols].apply(
                pd.to_numeric, errors='coerce'
            ).astype(np.float64)

            df_float = temp_df1[float_cols]
            valid = (~df_float.isna()) & (df_float != 0) & (df_float != 1) & (df_float % 1 != 0) # Exclude 0, 1, nan, and integers
            mask = valid.all(axis=1) # check for each row where all its colums are true.
            cleaned_df = temp_df1[mask].copy() # remove stations with at least one invalid value
            
            valid = valid.set_index(temp_df1['station'])
            stacked_valid = valid.stack()
            unvalid_locations = stacked_valid[~stacked_valid].index.tolist()
            if len(unvalid_locations) != 0:
                for item in unvalid_locations:
                    msg = f"Warning(Invalid_Value): IM {item[1]} at station {item[0]} for {cur_im_file}  at version {v} is invalid and removed."
                    log_and_print(log_lines, msg)
            
            # identifying missing IMs
            for col in im_true_order:
                if col not in cleaned_df.columns:
                    msg = f"Warning(Missed_IM): The IM {col} does not exist in the {cur_im_file} "
                    log_and_print(log_lines, msg)
                    # cleaned_df[col] = np.nan
                    
            #filtering only the component of interest
            filtered_im_df = cleaned_df[["station", "component"] + im_true_order]
            filtered_im_df = filtered_im_df[filtered_im_df["component"] == Comp].copy()
            if filtered_im_df.empty:
                msg = f"Warning(Empty_Filtered_File): {cur_im_file} in version {v} is empty after filtering. Maybe no valid IMs or Component {Comp} found."
                log_and_print(log_lines, msg)
            
            # write data to database
            REL_DATA[cur_im_file.stem] = filtered_im_df
            
        IM_DATA[v] = REL_DATA
    
    # identify common sations for a fault    
    common_stations = set.intersection(
        *[set(df["station"]) for rel_data in IM_DATA.values() for df in rel_data.values() if df is not None ]
    )
 
    if len(common_stations) == 0:
        msg = f"Warning(All_Reals_Common_Station): Realizations of {cur_fault} do not share common stations among diffferent versions. "
        log_and_print(log_lines, msg)
    
    # filtering only the common stations 
    common_columns = set(im_true_order)
    for v in cur_fault_versions:
        for rel in IM_DATA[v].keys():
            tempdf = IM_DATA[v][rel]
            tempdf = tempdf[tempdf['station'].isin(common_stations)].copy()
            if len(tempdf) == 0:
                msg = f"Warning(Single_Real_Common_Station): The realization {rel} of fault {cur_fault} in version {v} lost all its data while combining as it does not share common stations with other realizations."
                log_and_print(log_lines, msg)
            IM_DATA[v][rel] = tempdf
    
    # identify common IM values 
    common_columns = set(im_true_order).intersection(
        *[set(df.columns) for rel_data in IM_DATA.values() for df in rel_data.values() if df is not None ]
    )
    common_ims_list = sorted(list(common_columns), key=im_true_order.index)
    correct_columns = ['station', 'component'] + common_ims_list       
    
    # filtering only the common columns
    for v in cur_fault_versions:
        for rel in IM_DATA[v].keys():
            tempdf = IM_DATA[v][rel]
            tempdf = tempdf[correct_columns].copy()
            IM_DATA[v][rel] = tempdf
    
    # writing IM to new files
    os.makedirs(os.path.join(combined_cs_files_path, cur_fault, 'IM'))
    file_counter = 0
    for v in cur_fault_versions:
        for rel in IM_DATA[v].keys():
            file_counter += 1
            tempdf = IM_DATA[v][rel]
            target_im_file = os.path.join(combined_cs_files_path, cur_fault, 'IM', f"{cur_fault}_REL{file_counter:02d}.csv")
            tempdf.to_csv(target_im_file, index=False)
            msg = f"Note(Version_Control):  {v}/{rel}  -->  {cur_fault}_REL{file_counter:02d}.csv"
            log_lines.append(msg)
            if file_counter == 1:
                summary_data.loc[cur_fault, "n_stations_combined"] = len(set(tempdf['station']))
                summary_data.loc[cur_fault, "n_IMs_combined"] = len(list(set(tempdf.columns) - {'station', 'component'}))
    
    summary_data.loc[cur_fault, "n_rel_combined"] = len([rel for v in cur_fault_versions for rel in IM_DATA[v]])
    
    ###############################################################################
    ########################### working on Median files ###########################
    ###############################################################################
    
    # create median file
    all_dfs = [
        df.set_index("station").drop("component", axis=1) for v in IM_DATA.values()
        for df in v.values()
        if df is not None and not df.empty
    ]

    ref_index = all_dfs[0].index
    ref_columns = all_dfs[0].columns
    all_dfs_aligned = [
        df.reindex(index=ref_index, columns=ref_columns).astype(np.float64)
        for df in all_dfs
    ]

    array_stack = np.stack([df.to_numpy() for df in all_dfs_aligned], axis=0)
    median_array = np.nanmedian(array_stack, axis=0)
    
    median_df = pd.DataFrame(median_array, index=ref_index, columns=ref_columns)
    median_df.insert(0, "station", median_df.index)  # Insert 'station' as the first column
    median_df.insert(1, "component", Comp)         # Add 'component' column with Comp value
    # write the median DataFrame to the CSV file
    median_im_file = os.path.join(combined_cs_files_path, cur_fault, 'IM', f"{cur_fault}.csv")
    median_df.to_csv(median_im_file, index=False)     
    
    ###############################################################################
    ########################### working on Source files ###########################
    ###############################################################################     
    
    os.makedirs(os.path.join(combined_cs_files_path, cur_fault, 'Source'))
    file_counter = 0

    for v in cur_fault_versions:
        # check if the source and IM files are one-to-one
        fault_version_source = [rel for rel in Path(Path(vers_path[v]) / cur_fault / "Source").rglob("*REL*.csv") if ".pertb." not in rel.name]
        fault_version_source.sort()
        realizations_im_files = [r.stem for r in fault_version_realizations]
        realizations_sr_files = [r.stem for r in fault_version_source]
        
        only_in_im = set(realizations_im_files) - set(realizations_sr_files)
        only_in_sr = set(realizations_sr_files) - set(realizations_im_files)
        
        if len(only_in_im) != 0:
            for item in only_in_im:
                msg = f"Warning(IM_without_Source): Realization {item} in {v}/{cur_fault} does not have source file!"
                log_lines.append(msg)
        
        if len(only_in_sr) != 0:
            for item in only_in_sr:
                msg = f"Warning(Source_without_IM): Source {item} in {v}/{cur_fault} does not have IM file!"
                log_lines.append(msg)
        
        # check if the info and IM files are one-to-one
        fault_version_info = [rel for rel in Path(Path(vers_path[v]) / cur_fault / "Source").rglob("*REL*.info")]
        fault_version_info.sort()
        realizations_info_files = [r.stem for r in fault_version_info]

        only_in_im_vs_info = set(realizations_im_files) - set(realizations_info_files)
        only_in_info_vs_im = set(realizations_info_files) - set(realizations_im_files)

        if len(only_in_im_vs_info) != 0:
            for item in only_in_im_vs_info:
                msg = f"Warning(IM_without_Info): Realization {item} in {v}/{cur_fault} does not have info file!"
                log_lines.append(msg)

        if len(only_in_info_vs_im) != 0:
            for item in only_in_info_vs_im:
                msg = f"Warning(Info_without_IM): Info {item} in {v}/{cur_fault} does not have IM file!"
                log_lines.append(msg)

        # check if the info and IM files are one-to-one
        fault_version_pertb = [rel for rel in Path(Path(vers_path[v]) / cur_fault / "Source").rglob("*REL*.pertb.csv")]
        fault_version_pertb.sort()
        realizations_pertb_files = [r.stem.removesuffix('.pertb') for r in fault_version_pertb]

        only_in_im_vs_pertb = set(realizations_im_files) - set(realizations_pertb_files)
        only_in_pertb_vs_im = set(realizations_pertb_files) - set(realizations_im_files)

        if len(only_in_im_vs_pertb) != 0:
            for item in only_in_im_vs_pertb:
                msg = f"Warning(IM_without_Pertb): Realization {item} in {v}/{cur_fault} does not have pertb file!"
                log_lines.append(msg)

        if len(only_in_pertb_vs_im) != 0:
            for item in only_in_pertb_vs_im:
                msg = f"Warning(Pertb_without_IM): Pertb {item} in {v}/{cur_fault} does not have IM file!"
                log_lines.append(msg)
                
        # check if median source file exists
        if not Path(Path(vers_path[v]) / cur_fault / "Source" / f"{cur_fault}.csv").exists():
            msg = f"Warning(Missing_source_mediam_file): Source median file does not exist for {v}/{cur_fault} !"
            log_lines.append(msg)
        
        # check if vm_params.yaml file exists
        if not Path(Path(vers_path[v]) / cur_fault / "Source" / "vm_params.yaml").exists():
            msg = f"Warning(Missing_vm_params_file): vm_params.yaml file does not exist for {v}/{cur_fault} !"
            log_lines.append(msg)
            
        # check if nzvm.cfg file exists
        if not Path(Path(vers_path[v]) / cur_fault / "Source" / "nzvm.cfg").exists():
            msg = f"Warning(Missing_nzvm_config_file): nzvm.cfg file does not exist for {v}/{cur_fault} !"
            log_lines.append(msg)
        

        src_folder = Path(vers_path[v]) / cur_fault / "Source"
        dst_folder = Path(combined_cs_files_path) / cur_fault / "Source"
        # copy source file   
        for item in realizations_sr_files:
            src = src_folder / f"{item}.csv"
            dst = dst_folder / f"{item}.csv"
            if src.exists():
                shutil.copy2(src, dst)
                
        # copy pertb file    
        for item in realizations_pertb_files:
            original_path = os.path.join(vers_path[v], cur_fault, "Source", f"{item}.pertb.csv")
            destination_path = os.path.join(combined_cs_files_path, cur_fault, "Source", f"{item}.pertb.csv")
            shutil.copy2(original_path, destination_path)
        
        # copy info file    
        for item in realizations_info_files:
            src = src_folder / f"{item}.info"
            dst = dst_folder / f"{item}.info"
            if src.exists():
                shutil.copy2(src, dst)
              
        # copy median source file if exists
        src = Path(vers_path[v]) / cur_fault / "Source" / f"{cur_fault}.csv"
        dst = Path(combined_cs_files_path) / cur_fault / "Source" / f"{cur_fault}.csv"
        if src.exists():
            shutil.copy2(src, dst)
            
        # copy median info file if exists
        src = Path(vers_path[v]) / cur_fault / "Source" / f"{cur_fault}.info"
        dst = Path(combined_cs_files_path) / cur_fault / "Source" / f"{cur_fault}.info"
        if src.exists():
            shutil.copy2(src, dst)
        
        # check if vm_params.yaml file exists
        src = Path(vers_path[v]) / cur_fault / "Source" / "vm_params.yaml"
        dst = Path(combined_cs_files_path) / cur_fault / "Source" / f"vm_params_from_cs100_{v}.yaml"
        if src.exists():
            shutil.copy2(src, dst)
            
        # check if nzvm.cfg file exists
        src = Path(vers_path[v]) / cur_fault / "Source" / "nzvm.cfg"
        dst = Path(combined_cs_files_path) / cur_fault / "Source" / f"nzvm_from_cs100_{v}.cfg"
        if src.exists():
            shutil.copy2(src, dst)
             
    
with open(log_file, 'w') as f:
    f.write("\n".join(log_lines))

  0%|          | 0/141 [00:00<?, ?it/s]

 21%|██        | 29/141 [08:01<29:06, 15.60s/it] /tmp/ipykernel_3907238/1150536705.py:98: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  cur_im_df = pd.read_csv(
 82%|████████▏ | 116/141 [36:41<07:01, 16.85s/it] /tmp/ipykernel_3907238/1150536705.py:98: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  cur_im_df = pd.read_csv(
100%|██████████| 141/141 [44:07<00:00, 18.78s/it]


# Create Report

In [38]:
# Save to CSV
station_im_summary_file_path = os.path.join(combined_cs_files_path, "version_station_im_summary.csv")
summary_data.to_csv(station_im_summary_file_path)
print(f"Stations and IMs data saved to {station_im_summary_file_path}")

# Write Note
Current_Date_and_Time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
note_file_path = os.path.join(combined_cs_files_path, 'note.txt')
[f"{v}, " for v in versions]
note_content = (
    f"- This folder contains combined IM (Intensity Measure) data from the realizations of the following versions:\n"
    f"[{', '.join(f'\"{v}\"' for v in versions)}]\n"
    f"- Date of generation: {Current_Date_and_Time}\n"
    f"- Key Notes:\n"
    f"    1. For each fault (source), only the common stations and IM types between the realizations (even differne tversions) have been retained.\n"
    f"    2. Median IM values for each station and IM type have been computed and saved as a separate CSV file.\n"
    f"    3. Source files have been copied:\n"
    f"       - 'nzvm.cfg' and 'vm_params.yaml' from each version are copied with a '_from_cs100_version' suffix.\n"
)


# Write to file
with open(note_file_path, "w") as file:
    file.write(note_content)

print(f"Note written to {note_file_path}")

Stations and IMs data saved to /mnt/hypo_data/mab419/cs100/Cybershake_Data/combined_cs100/version_station_im_summary.csv
Note written to /mnt/hypo_data/mab419/cs100/Cybershake_Data/combined_cs100/note.txt
